In [52]:
import pandas as pd
import re
list_csv = ['data/kor/attraction_places.csv',
            'data/kor/hotel_places.csv',
            'data/kor/restaurant_places.csv',
            'data/eng/eng_attraction_places.csv',
            'data/eng/eng_hotel_places.csv',
            'data/eng/eng_restaurant_places.csv']
save_csv = ['data/attraction_places.csv',
            'data/hotel_places.csv',
            'data/restaurant_places.csv']
save2_csv = ['data/attraction_dropped.csv',
             'data/hotel_dropped.csv',
             'data/restaurant_dropped.csv']

In [53]:
for csv in list_csv:
    df = pd.read_csv(csv,
                     names=['placeId', 'name_kor', 'location', 'class'],
                     encoding='euc-kr')
    print(len(df))

561
804
23150
564
899
12599


In [54]:
matches = re.findall('[0-9]+\.[0-9]+', str1)
t = tuple([float(s) for s in matches])

In [55]:
def tuplize(arg):
    try:
        return tuple([float(s) for s in arg])
    except:
        print(arg)

In [60]:
for i in range(3):
    df_kor = pd.read_csv(list_csv[i],
                     names=['placeId', 'name_kor', 'location', 'class'],
                     encoding='euc-kr')
    tmp_class = df_kor['class'][0]
    df_eng = pd.read_csv(list_csv[i+3],
                     names=['placeId', 'name_eng', 'location', 'class'],
                     encoding='euc-kr')
    df_eng = df_eng.iloc[:, [j for j, c in enumerate(df_eng.columns) if j != 4]]
    df_eng = df_eng.drop(columns=['location', 'class'])
    df_kor = pd.merge(df_kor, df_eng, how='outer')
    cols = df_kor.columns.tolist()
    cols = cols[3:4] + cols[0:2] + cols[-1:] + cols[2:3]
    df_kor = df_kor[cols]
    df_kor['class'] = df_kor['class'].fillna(tmp_class)

    # 테스트용: 좌표 외 drop
    df_dropped = df_kor.dropna(subset=['location'])
    series_match = df_dropped['location'].apply(lambda x:
                                                re.fullmatch('[0-9]+\.[0-9]+,[0-9]+\.[0-9]+', x))
    df_dropped = df_dropped[pd.notnull(series_match)]
    
    # 좌표 (float,float)형 tuple로 변환
    df_location = pd.DataFrame(df_dropped['location'].apply(lambda x:
                                                            re.findall('[0-9]+\.[0-9]+', x)))
    df_dropped['latitude'] = df_location['location'].apply(lambda x: x[0])
    df_dropped['longitude'] = df_location['location'].apply(lambda x: x[0])
    ###
    df_kor.to_csv(save_csv[i], index=False)
    df_dropped.to_csv(save2_csv[i], index=False)
    print('before drop: ', len(df_kor))
    print('after  drop: ', len(df_dropped))
    print()

before drop:  569
after  drop:  352

before drop:  1163
after  drop:  687

before drop:  23171
after  drop:  18250



In [65]:
df_dropped

,class,placeId,name_kor,name_eng,location,latitude
3,restaurants,10000599,빌즈 광화문,bills Gwanghwamun,"37.5717,126.97971",37.5717
5,restaurants,10025748,뱀부하우스,Bamboo House,"37.507,127.03664",37.507
6,restaurants,10040756,117 파크,117 Park,"37.571552,126.970795",37.571552
10,restaurants,1006353,레보,Les Baux,"37.52602,127.03659",37.52602
11,restaurants,10063988,참치공방,Chamchi Gongbang,"37.57072,126.98422",37.57072
12,restaurants,10065566,할스,Hearth,"37.50355,127.03987",37.50355
13,restaurants,10073597,오리지널 팬케이크 하우스,The Original Pancake House Itaewon,"36.951187,127.02991",36.951187
17,restaurants,10079161,BHC치킨 명동본점,BHC Chicken Myeongdong,"37.564575,126.983864",37.564575
18,restaurants,10080270,아카시아 카페 비스트로,Acacia Cafe & Bistro,"37.5199,127.022675",37.5199
19,restaurants,10083141,탑 카페테리아,TOP cafeteria,"37.563343,127.0371",37.563343


In [51]:
print('전체',len(df_kor))
print('NA',len(df_kor[pd.isna(df_kor['location'])]))
df_tmp = df_kor.dropna(subset=['location'])
series_match = df_tmp['location'].apply(lambda x:
                                        re.fullmatch('[0-9]+\.[0-9]+,[0-9]+\.[0-9]+', x))
print('좌표',len(df_tmp[pd.notnull(series_match)]))
print('주소',len(df_tmp[pd.isnull(series_match)]))

전체 23171
NA 26
좌표 18250
주소 4895
